<a href="https://colab.research.google.com/github/quil-aditya/offline-meeting-transcriber/blob/main/SynapseScribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Block -1 : transcript and diarization**

 1. Install Required Libraries

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install torch torchaudio transformers accelerate
!pip install librosa


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0qtefv8h
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0qtefv8h
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install torch torchaudio librosa resemblyzer scikit-learn


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-x7mo6tyf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-x7mo6tyf
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 73.0 MB/s eta 0:00:00
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-linux_x86_64.whl size=73500 sha256=058a31f02dde21939d91a9bfa069d958125095c439e1ce87581cc09780edfad1
  Stored in directory: /root/.cache/pip/wheels/94/65/3f/292d0b656be33d1c801

2. Upload audio

In [ ]:
from google.colab import files
uploaded = files.upload()

import os
audio_path = next(iter(uploaded))
print(f"📂 Audio file loaded: {audio_path}")


Saving Comicstorian-episode-_Bruce Wayne vs Alfred Pennyworth_ Absolute Batman.mp3 to Comicstorian-episode-_Bruce Wayne vs Alfred Pennyworth_ Absolute Batman.mp3
📂 Audio file loaded: Comicstorian-episode-_Bruce Wayne vs Alfred Pennyworth_ Absolute Batman.mp3


3: Transcribe Audio Using Whisper

In [ ]:
import whisper

model = whisper.load_model("base")  # or "small" if GPU available
result = model.transcribe(audio_path, word_timestamps=True)

print("✅ Transcription complete")
print(result["text"][:500])  # Preview first 500 characters
segments = result["segments"]  # We'll use this later


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 143MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Transcription complete
 Without his mansion, money, or butler, who is the absolute Batman? Welcome to the comic story and complete story series where we take trade paperbacks and single issues and we break them down into digestible bites to help you understand. Then we read them dramatically back to you. All alterations to the panels, text, and images to prevent copyright problems and all art is owned by its respective companies. Absolute DC has begun starting with Absolute Batman, written by Scott Snyder. A version o


4: Prepare for Speaker Diarization with Resemblyzer

In [ ]:
import librosa
import numpy as np
from resemblyzer import VoiceEncoder
from resemblyzer.hparams import sampling_rate

# Load audio as mono @ 16kHz
wav, sr = librosa.load(audio_path, sr=sampling_rate)
encoder = VoiceEncoder()

# Slice audio into overlapping chunks (1.5s window, 0.75s hop)
window = int(1.5 * sampling_rate)
hop = int(0.75 * sampling_rate)
slices = [wav[i:i+window] for i in range(0, len(wav) - window, hop)]

# Get speaker embeddings
embeddings = np.array([encoder.embed_utterance(s) for s in slices])
print(f"🧬 Generated {len(embeddings)} embeddings")


/tmp/ipython-input-3-3823177820.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(audio_path, sr=sampling_rate)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Loaded the voice encoder model on cpu in 0.03 seconds.
🧬 Generated 1592 embeddings


5: Cluster Speakers Using KMeans

In [ ]:
from sklearn.cluster import KMeans

# Choose number of speakers (manual for now — later we can auto-tune)
n_speakers = 2  # Adjust if needed
kmeans = KMeans(n_clusters=n_speakers, random_state=0).fit(embeddings)
labels = kmeans.labels_

print(f"🗣️ Assigned speaker labels: {set(labels)}")


🗣️ Assigned speaker labels: {np.int32(0), np.int32(1)}


6: Convert Labels to Timestamped Speaker Segments

In [ ]:
# Each slice is 1.5s, overlapped every 0.75s
speaker_segments = []
for i, label in enumerate(labels):
    start_time = i * 0.75
    end_time = start_time + 1.5
    speaker_segments.append({
        "start": start_time,
        "end": end_time,
        "speaker": f"Speaker {label + 1}"
    })

# Optional: Print a few segments
for s in speaker_segments[:5]:
    print(s)


{'start': 0.0, 'end': 1.5, 'speaker': 'Speaker 1'}
{'start': 0.75, 'end': 2.25, 'speaker': 'Speaker 1'}
{'start': 1.5, 'end': 3.0, 'speaker': 'Speaker 1'}
{'start': 2.25, 'end': 3.75, 'speaker': 'Speaker 2'}
{'start': 3.0, 'end': 4.5, 'speaker': 'Speaker 1'}


7: Match Speakers to Whisper Segments (Simple Mapping)

In [ ]:
def find_speaker_for_time(t, segments):
    for seg in segments:
        if seg["start"] <= t <= seg["end"]:
            return seg["speaker"]
    return "Unknown"

# Assign speaker to each Whisper segment
for seg in result["segments"]:
    mid_time = (seg["start"] + seg["end"]) / 2
    speaker_label = find_speaker_for_time(mid_time, speaker_segments)
    seg["speaker"] = speaker_label

# Show sample output
for seg in result["segments"][:5]:
    print(f'{seg["speaker"]}: {seg["text"]}')


Speaker 1:  Without his mansion, money, or butler, who is the absolute Batman?
Speaker 1:  Welcome to the comic story and complete story series where we take trade paperbacks and single
Speaker 1:  issues and we break them down into digestible bites to help you understand. Then we read them
Speaker 1:  dramatically back to you. All alterations to the panels, text, and images to prevent copyright
Speaker 1:  problems and all art is owned by its respective companies.


In [ ]:
from itertools import groupby

merged_transcript = []

# Group segments where the speaker stays the same
for speaker, group in groupby(result["segments"], key=lambda x: x["speaker"]):
    texts = [seg["text"].strip() for seg in group]
    full_text = " ".join(texts)
    merged_transcript.append({
        "speaker": speaker,
        "text": full_text
    })

# Output cleaned transcript
for block in merged_transcript:
    print(f'{block["speaker"]}: {block["text"]}\n')


Speaker 1: Without his mansion, money, or butler, who is the absolute Batman? Welcome to the comic story and complete story series where we take trade paperbacks and single issues and we break them down into digestible bites to help you understand. Then we read them dramatically back to you. All alterations to the panels, text, and images to prevent copyright problems and all art is owned by its respective companies. Absolute DC has begun starting with Absolute Batman, written by Scott Snyder.

Speaker 2: A version of Bruce Wayne without the Wayne fortune and all that came with it. Today, we get a glimpse

Speaker 1: at his origin and see how he handles the gangs rampaging through this new Gotham. Anyway, I hope you enjoy. Sometime in the past, a young Bruce Wayne leans over the sign for the Bat House. His eyes widened in fascination with the mural. But he turns when his father calls out to him. Thomas Wayne gathers his class at the Gotham Zoo, informing his son that there will be time

# **Block -2 : Emotion detection**

Requirements:

In [ ]:
!pip install transformers torch librosa


A: Install & Load Emotion Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

emotion_tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
emotion_model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base").to(device)
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

B: (If needed) Translate Hindi → English

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load Hindi to English translation model
trans_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-hi-en")
trans_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-hi-en").to(device)

def translate_hi_to_en(text):
    inputs = trans_tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    translated = trans_model.generate(**inputs)
    return trans_tokenizer.decode(translated[0], skip_special_tokens=True)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/304M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/304M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

C: Emotion Detection Function



In [ ]:
def detect_emotion(text):
    # Ensure it's in English
    if any('\u0900' <= c <= '\u097F' for c in text):  # Detect Hindi script
        text = translate_hi_to_en(text)

    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = emotion_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    emotion = emotion_labels[torch.argmax(probs).item()]
    return emotion


D: Attach Emotion to Each Speaker Block

In [ ]:
for block in merged_transcript:
    emotion = detect_emotion(block["text"])
    block["emotion"] = emotion
    print(f'{block["speaker"]} ({emotion}): {block["text"]}\n')


Speaker 1 (neutral): Without his mansion, money, or butler, who is the absolute Batman? Welcome to the comic story and complete story series where we take trade paperbacks and single issues and we break them down into digestible bites to help you understand. Then we read them dramatically back to you. All alterations to the panels, text, and images to prevent copyright problems and all art is owned by its respective companies. Absolute DC has begun starting with Absolute Batman, written by Scott Snyder.

Speaker 2 (neutral): A version of Bruce Wayne without the Wayne fortune and all that came with it. Today, we get a glimpse

Speaker 1 (neutral): at his origin and see how he handles the gangs rampaging through this new Gotham. Anyway, I hope you enjoy. Sometime in the past, a young Bruce Wayne leans over the sign for the Bat House. His eyes widened in fascination with the mural. But he turns when his father calls out to him. Thomas Wayne gathers his class at the Gotham Zoo, informing h

# **Block -3: Summarization**

1. Install Requirements

In [ ]:
!pip install transformers


2. Load BART Summarizer Model

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer_summ = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model_summ = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

3. Merge All Text (if not done yet)

In [ ]:
full_text = " ".join([block["text"] for block in merged_transcript])


4. Split Text into Chunks (if > 1024 tokens)

In [ ]:
def split_into_chunks(text, max_tokens=1024):
    words = text.split()
    return [" ".join(words[i:i+max_tokens]) for i in range(0, len(words), max_tokens)]

chunks = split_into_chunks(full_text, max_tokens=900)  # 900 for safety


 5. Summarize Each Chunk and print final


In [ ]:
summaries = []

for chunk in chunks:
    inputs = tokenizer_summ.batch_encode_plus(
        [chunk],
        max_length=1024,
        truncation=True,
        padding="longest",
        return_tensors="pt"
    ).to(device)

    summary_ids = model_summ.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=150,
        min_length=30,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    summary_text = tokenizer_summ.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary_text)

final_summary = "\n".join(summaries)
print("📝 Final Summary:\n", final_summary)


📝 Final Summary:
 Absolute DC has begun starting with Absolute Batman, written by Scott Snyder. A version of Bruce Wayne without the Wayne fortune and all that came with it. Today, we get a glimpse at his origin and see how he handles the gangs rampaging through this new Gotham.
Alfred watches as a large group of party animals approach the town hall. Bruce Wayne stands upon a rooftop, pulling on his cowl and leaping off into the night air. The next day, the party animals continue their reign of chaos throughout the city.
Alfred follows the young man as he moves around the city and learns all he can about him. Bruce Wayne grew up in crime alley. His mother is a social worker. His father, a teacher. Bruce was smart, excelled in school. But his life truly began when his class took a trip to the Gotham Zoo.
We lost him somewhere in the Philippines. He's in the wind. Meanwhile, in Manila, two helicopter pilots are preparing for a flight. Right before their boss slides open the helicopter do

# **Block -4 : MoM generation**

Smart MoM Extractor Function

In [ ]:
import re

def extract_mom(summary_text):
    lines = summary_text.strip().split('. ')
    mom = {"decisions": [], "actions": [], "notes": []}

    for line in lines:
        line = line.strip(". ").strip()

        if not line:
            continue

        lower = line.lower()

        # 🔹 Action items
        if any(k in lower for k in ["will", "need to", "must", "should", "required", "responsible", "assigned", "has to", "expected to"]):
            if re.search(r"\b(by|before|on|within)\b.*\b(monday|tuesday|friday|week|month|\d{1,2})", lower):
                line += " [📅 Deadline mentioned]"
            mom["actions"].append(line)

        # 🔹 Decisions
        elif any(k in lower for k in ["decided", "approved", "agreed", "confirmed", "finalized", "accepted", "declined"]):
            mom["decisions"].append(line)

        # 🔹 Fallback Notes
        else:
            mom["notes"].append(line)

    return mom


Print a Professional MoM Output

In [ ]:
def print_mom(mom):
    print("\n📝📋 Final Minutes of Meeting (Auto-Generated)\n")

    if mom["actions"]:
        print("🔸 ACTION ITEMS:\n")
        for i, item in enumerate(mom["actions"], 1):
            print(f"  {i}. {item}")

    if mom["decisions"]:
        print("\n✅ DECISIONS TAKEN:\n")
        for i, item in enumerate(mom["decisions"], 1):
            print(f"  {i}. {item}")

    if mom["notes"]:
        print("\n🗒️ GENERAL DISCUSSION NOTES:\n")
        for i, item in enumerate(mom["notes"], 1):
            print(f"  {i}. {item}")


Run the MoM Generator

In [ ]:
# Extract & Display MoM
mom = extract_mom(final_summary)
print_mom(mom)



📝📋 Final Minutes of Meeting (Auto-Generated)


🗒️ GENERAL DISCUSSION NOTES:

  1. Absolute DC has begun starting with Absolute Batman, written by Scott Snyder
  2. A version of Bruce Wayne without the Wayne fortune and all that came with it
  3. Today, we get a glimpse at his origin and see how he handles the gangs rampaging through this new Gotham.
Alfred watches as a large group of party animals approach the town hall
  4. Bruce Wayne stands upon a rooftop, pulling on his cowl and leaping off into the night air
  5. The next day, the party animals continue their reign of chaos throughout the city.
Alfred follows the young man as he moves around the city and learns all he can about him
  6. Bruce Wayne grew up in crime alley
  7. His mother is a social worker
  8. His father, a teacher
  9. Bruce was smart, excelled in school
  10. But his life truly began when his class took a trip to the Gotham Zoo.
We lost him somewhere in the Philippines
  11. He's in the wind
  12. Meanwhile, in

Detailed MoM from result["segments"]

In [ ]:
def generate_detailed_mom(segments):
    detailed = []

    for seg in segments:
        start = seg["start"]
        end = seg["end"]
        speaker = seg.get("speaker", "Unknown")
        text = seg["text"].strip()
        flags = []

        # Check for action/decision keywords
        if any(k in text.lower() for k in ["will", "must", "need to", "required", "should", "has to"]):
            flags.append("📌 Action")
        if any(k in text.lower() for k in ["decided", "approved", "agreed", "confirmed"]):
            flags.append("✅ Decision")
        if re.search(r"\b(by|before|on|within)\b.*\b(monday|tuesday|week|month|\d{1,2})", text.lower()):
            flags.append("📅 Deadline")

        timestamp = f"[{start:.2f} – {end:.2f}]"
        flag_str = f" {' '.join(flags)}" if flags else ""
        detailed.append(f"{timestamp} {speaker}: {text}{flag_str}")

    return detailed


Display the Detailed MoM

In [ ]:
def print_detailed_mom(detailed_lines):
    print("\n🕰️ DETAILED MINUTE-WISE MOM:\n")
    for line in detailed_lines:
        print(line)


Run Both Formats Together

In [ ]:
# Condensed MoM
mom = extract_mom(final_summary)
print_mom(mom)

# Detailed MoM
detailed_lines = generate_detailed_mom(result["segments"])
print_detailed_mom(detailed_lines)



📝📋 Final Minutes of Meeting (Auto-Generated)


🗒️ GENERAL DISCUSSION NOTES:

  1. Absolute DC has begun starting with Absolute Batman, written by Scott Snyder
  2. A version of Bruce Wayne without the Wayne fortune and all that came with it
  3. Today, we get a glimpse at his origin and see how he handles the gangs rampaging through this new Gotham.
Alfred watches as a large group of party animals approach the town hall
  4. Bruce Wayne stands upon a rooftop, pulling on his cowl and leaping off into the night air
  5. The next day, the party animals continue their reign of chaos throughout the city.
Alfred follows the young man as he moves around the city and learns all he can about him
  6. Bruce Wayne grew up in crime alley
  7. His mother is a social worker
  8. His father, a teacher
  9. Bruce was smart, excelled in school
  10. But his life truly began when his class took a trip to the Gotham Zoo.
We lost him somewhere in the Philippines
  11. He's in the wind
  12. Meanwhile, in